In [1]:
%load_ext autoreload
%autoreload 2
import os 
import pandas as pd
import numpy as np
os.chdir('..')
from src.data.make_dataset import get_preprocessed_datasets
from src.features.build_features import FeatureEngineering

In [2]:
df_bitcoin, df, df_test = get_preprocessed_datasets()

2019-11-03 20:29:10,162 - src.data.make_dataset - INFO - making final data sets from raw data
2019-11-03 20:29:10,400 - src.data.make_dataset - INFO - Shape of df_bitcoin: (2355, 2)
2019-11-03 20:29:10,401 - src.data.make_dataset - INFO - Shape of df: (4757, 91)
2019-11-03 20:29:10,456 - src.data.make_dataset - INFO - Start preprocessing dataframe
2019-11-03 20:29:10,483 - src.data.make_dataset - INFO - Replacing market_data_current_price_usd 
2019-11-03 20:29:10,507 - src.data.make_dataset - INFO - Replacing market_data_ath_usd 
2019-11-03 20:29:10,524 - src.data.make_dataset - INFO - Replacing market_data_circulating_supply 
2019-11-03 20:29:10,543 - src.data.make_dataset - INFO - Preprocessing done!


In [3]:
fe = FeatureEngineering(df, df_bitcoin, df_test)
fe.construct_features()
fe.df_features.head()

/c/Users/Sandro/repo/data_science_for_business_lmu/src/features/build_features.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.df = pd.concat([df, df_test])
2019-11-03 20:29:10,723 - src.features.build_features - INFO - Adding categories_0
2019-11-03 20:29:10,754 - src.features.build_features - INFO - Transform numerical variable for column transaction_count
2019-11-03 20:29:10,757 - src.features.build_features - INFO - Start filling NA values in transaction_count
2019-11-03 20:29:10,768 - src.features.build_features - INFO - Found 2695 NA values in column transaction_count
2019-11-03 20:29:10,775 - src.features.build_features - INFO - Filled NA values
2019-11-03 20:29:10,799 - src.features.build_features - INFO - Transform numerical variable for column holder_cou

,OBS_ID,success,labels_categories_0,transaction_count,holder_count,market_data_current_price_usd,market_data_ath_usd,market_data_total_supply,market_data_circulating_supply,public_interest_stats_bing_matches
0,235691,0,23,48262.141038,13576.305982,0.001677,0.034182,6.565882e+07,8.627700e+13,23900.0
1,235692,1,29,975773.000000,109840.000000,0.231561,43587.000000,1.000000e+09,6.008030e+15,686000.0
2,235693,0,29,22730.000000,13914.000000,0.000717,0.043750,5.000000e+08,4.400000e+08,27500.0
3,235694,0,26,125.000000,51.000000,0.033603,43505.000000,5.000000e+06,0.000000e+00,27500.0
4,235695,0,26,16417.000000,6250.000000,0.000887,0.030843,1.000000e+07,0.000000e+00,703000.0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [8]:
X_train, y_train, X_test = fe.get_X_y()

X_train = X_train.values
y_train = y_train.values.astype(int)

In [9]:
X_train

array([[2.3000000e+01, 4.8262141e+04, 1.3576306e+04, ..., 6.5658824e+07,
        8.6277000e+13, 2.3900000e+04],
       [2.9000000e+01, 9.7577300e+05, 1.0984000e+05, ..., 1.0000000e+09,
        6.0080300e+15, 6.8600000e+05],
       [2.9000000e+01, 2.2730000e+04, 1.3914000e+04, ..., 5.0000000e+08,
        4.4000000e+08, 2.7500000e+04],
       ...,
       [2.6000000e+01, 4.8262141e+04, 1.3576306e+04, ..., 1.2750000e+08,
        0.0000000e+00, 8.9000000e+04],
       [2.6000000e+01, 1.6800000e+02, 3.4000000e+01, ..., 2.0000000e+11,
        0.0000000e+00, 8.2800000e+04],
       [2.3000000e+01, 4.8262141e+04, 1.3576306e+04, ..., 3.0000000e+08,
        0.0000000e+00, 2.7400000e+04]])

In [10]:
y_train

array([0, 1, 0, ..., 0, 0, 0])

In [11]:
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)

X_val = X_val.astype(int)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3805, 8)
(3805,)
(952, 8)
(952,)


In [13]:
y_train = y_train.astype(int)

In [20]:
model = RandomForestClassifier(n_estimators=1000)

In [21]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
y_preds = model.predict(X_val)

In [23]:
from sklearn.metrics import matthews_corrcoef


In [24]:
y_val = y_val.astype(int)

In [25]:
matthews_corrcoef(y_val, y_preds)

0.09148758026851055

## Test Set

In [26]:
X_train, y_train, X_test = fe.get_X_y()
X_train.head()

,labels_categories_0,transaction_count,holder_count,market_data_current_price_usd,market_data_ath_usd,market_data_total_supply,market_data_circulating_supply,public_interest_stats_bing_matches
0,23,48262.141038,13576.305982,0.001677,0.034182,6.565882e+07,8.627700e+13,23900.0
1,29,975773.000000,109840.000000,0.231561,43587.000000,1.000000e+09,6.008030e+15,686000.0
2,29,22730.000000,13914.000000,0.000717,0.043750,5.000000e+08,4.400000e+08,27500.0
3,26,125.000000,51.000000,0.033603,43505.000000,5.000000e+06,0.000000e+00,27500.0
4,26,16417.000000,6250.000000,0.000887,0.030843,1.000000e+07,0.000000e+00,703000.0


In [32]:
a = list(X_test.columns)

In [36]:
a.remove('OBS_ID')

In [37]:
a

['labels_categories_0',
 'transaction_count',
 'holder_count',
 'market_data_current_price_usd',
 'market_data_ath_usd',
 'market_data_total_supply',
 'market_data_circulating_supply',
 'public_interest_stats_bing_matches']

In [40]:
X_test_values = X_test[a].values

In [41]:
X_test_values

array([[2.60000000e+01, 3.76000000e+02, 2.65000000e+02, ...,
        2.85201000e+07, 0.00000000e+00, 1.09000000e+07],
       [2.60000000e+01, 7.04080000e+04, 2.31150000e+04, ...,
        4.00000000e+08, 4.01161660e+07, 2.86000000e+04],
       [2.60000000e+01, 4.82621410e+04, 1.35763060e+04, ...,
        2.14188959e+15, 2.92934000e+14, 4.26000000e+05],
       ...,
       [2.60000000e+01, 4.82621410e+04, 1.35763060e+04, ...,
        3.00000000e+07, 0.00000000e+00, 2.69000000e+04],
       [2.60000000e+01, 8.77800000e+04, 7.01990000e+04, ...,
        2.05000000e+09, 1.45883000e+16, 2.83000000e+04],
       [8.00000000e+00, 4.82621410e+04, 1.35763060e+04, ...,
        1.00000000e+10, 0.00000000e+00, 0.00000000e+00]])

In [42]:
preds_test = model.predict(X_test_values)

In [43]:
pd.DataFrame({"OBS_ID": X_test['OBS_ID'],
             "success": preds_test}).to_csv('data/processed/submission03', index=None)

In [45]:
a = pd.read_csv('data/processed/submission03')

In [54]:
b = pd.read_csv('data/processed/submission02')

In [53]:
a.success.value_caounts()

0    982
1     19
Name: success, dtype: int64

In [56]:
b.success.value_counts()

0    988
1     13
Name: success, dtype: int64